In [180]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from gtda.graphs import KNeighborsGraph
import itertools
from utils import GraphRNN_dataset, GraphRNN_DataSampler
import random

from importlib import reload
import matplotlib.pyplot as plt

from preprocessor import Preprocessor
from preprocessor import draw_network
from preprocessor import get_adj_from_plot

print("Gathering data...")
flow_dataset = "../data/daily_county2county_2019_01_01.csv"
epi_dataset = "../data_epi/epidemiology.csv"
epi_dates = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
             "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
             "2020-06-21", "2020-06-22", "2020-06-23", "2020-06-24",
                "2020-06-25", "2020-06-26", "2020-06-27", "2020-06-28",
                "2020-06-29", "2020-06-30", "2020-07-01", "2020-07-02",
                "2020-07-03", "2020-07-04", "2020-07-05", "2020-07-06",
                "2020-07-07", "2020-07-08", "2020-07-09", "2020-07-10",
                "2020-07-11", "2020-07-12", "2020-07-13", "2020-07-14",
                "2020-07-15", "2020-07-16", "2020-07-17", "2020-07-18",
                "2020-07-19", "2020-07-20", "2020-07-21", "2020-07-22",
                "2020-07-23", "2020-07-24", "2020-07-25", "2020-07-26",
                "2020-07-27", "2020-07-28", "2020-07-29", "2020-07-30"
             ]

epi_dates_smaller = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
             "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
             ]

epi_dates_one_sample = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15"] # for testing
epi_dates_pred_one_sample = ["2020-06-16"] # for testing

Gathering data...


In [171]:
# THIS IS WITH THE NEW METHOD

print("Preprocessing data...")

# Use this for training to choose what size of data you want to use
cur_epi_dates = epi_dates_smaller # only does this up until the last date
# cur_epi_dates = epi_dates 

# Preprocess the data
input_hor = 3   # test now with smaller input horizon
pred_hor = 1    # test now with smaller prediction horizon
train_perc, test_perc = 0.8, 0.2

print("Building the kronecker graph...")
preprocessor = Preprocessor(flow_dataset, epi_dataset, cur_epi_dates, plottable=True) # test first with epi dates smaller
graph_kronecker_whole_df = preprocessor.combined_manual_kronecker() # makes the pandas df from the kronecker data

# if the first date is not equal to the first date in the epi_dates, then raise an error
if graph_kronecker_whole_df["date_y"][0] != cur_epi_dates[0]:
   raise ValueError("ERROR: The first date in the pandas dataframe is not equal to the first date in the cur_epi_dates")

# if the last date is not equal to the last date in the epi_dates, then raise an error
if graph_kronecker_whole_df["date_y"].iloc[-1] != cur_epi_dates[-2]:
   raise ValueError("ERROR: The last date in the pandas dataframe is not equal to the last date in the cur_epi_dates")

# print last dataframe in graph_kronecker_whole_df, check if the data is getting pulled correctly
print("Check: Last dataframe in graph_kronecker_whole_df: ", graph_kronecker_whole_df.iloc[-1])

# print("Drawing the network ...")
# draw_network(graph_kronecker_whole_df)



print("Getting the adjacency matrix..")
adj_kronecker_whole = get_adj_from_plot(graph_kronecker_whole_df)




print("Getting the training graph signal...")
tr_epi = preprocessor.set_timestep_offset_epi_dataset(from_timestep=0).get_epi_dataset() # will get the epi info for the entire dataset, the entire kronecker, then later we index per training example and pred
train_graph_sig = {}
for data in tr_epi:
    train_graph_sig = {**train_graph_sig, **dict(zip(data.geoid_o, data.new_confirmed))}

# sort train_graph_sig by geoid_o
train_graph_sig = dict(sorted(train_graph_sig.items(), key=lambda item: item[0]))

# check if any of the values in the dictionary are Nan, if yes, print and replace with 0.0
for key, value in train_graph_sig.items():
    if np.isnan(value):
        print("Nan value found in train_graph_sig: ", key, value, ". Replacing with 0.0 ...")
        # replace NaN with 0.0
        train_graph_sig[key] = 0.0

# what are the last 10 elements in train_graph_sig? Is all the data gettting pulled?
print("Check: Last 10 elements in train_graph_sig: ", {k: train_graph_sig[k] for k in list(train_graph_sig)[-10:]}) # yes the data is getting pulled here

# check if train_graph_sig being pulled correctly
print("Check: Size of train_graph_sig: ", len(train_graph_sig)) 

# check if the last few elements in train_graph_sig are being pulled correctly 
print("Check: First few from train_graph_sig: ", {k: train_graph_sig[k] for k in list(train_graph_sig)[:5]})

# how many nodes are in the train_graph_sig across the whole kronecker? This is a check
print("Check: Number of nodes in the train_graph_sig: ", len(train_graph_sig))





# for the first 14329 edges, collect the unique geoid_o and geoid_d in one set
unique_geoids = set()

# TODO: remove this hardcoding on the number of edges if possible
for i in range(14329):
    geoid_o = graph_kronecker_whole_df.loc[i, 'geoid_o']
    unique_geoids.add(geoid_o)

print("Number of unique geoids: ", len(unique_geoids))






''' Testing for a single example

# now draw out the first adjacency matrix
print("Getting the first adjacency matrix...")
adj_per_example = adj_kronecker_whole[:len(unique_geoids)*input_hor, :len(unique_geoids)*input_hor] 

if adj_kronecker_whole[0, len(unique_geoids)*input_hor+1] != 0.0:
    raise ValueError("The value at adj_kronecker_whole[0, len(unique_geoids)*input_hor+1] is not equal to 0.0")


# now drawing out the first train_graph_signal
print("Getting the first train_graph_signal...")

# if the number of nodes in train_graph_sig is not equal to len(cur_epi_dates)*unique_geoids, then raise an error
if len(train_graph_sig) != len(cur_epi_dates)*len(unique_geoids):
    raise ValueError("The number of nodes in train_graph_sig is not equal to len(cur_epi_dates)*len(unique_geoids)")

# print("Training graph signal is", train_graph_sig)

# get the graph signal corresponding to the first set of nodes
train_graph_sig_per_example = {k: train_graph_sig[k] for k in list(train_graph_sig)[:len(unique_geoids)*input_hor]}

# print the last few examples in train_graph_sig_per_example - working! 
print("Check: Last few examples in train_graph_sig_per_example: ", {k: train_graph_sig_per_example[k] for k in list(train_graph_sig_per_example)[-5:]})

# check "the next one" after train_graph_sig_per_example -- working!
print("Check: The next one after train_graph_sig_per_example: ", {k: train_graph_sig[k] for k in list(train_graph_sig)[len(unique_geoids)*input_hor:len(unique_geoids)*input_hor+5]})


# get the graph signal corresponding to the [input_hor : input_hor+pred_hor] set of nodes -- working!
train_graph_sig_per_example_pred = {k: train_graph_sig[k] for k in list(train_graph_sig)[len(unique_geoids)*input_hor:len(unique_geoids)*(input_hor+pred_hor)]}

print("Check: Last few examples in train_graph_sig_pred: ", {k: train_graph_sig_per_example_pred[k] for k in list(train_graph_sig_per_example_pred)[-5:]}) # working!

training_example = [adj_per_example, train_graph_sig_per_example, train_graph_sig_per_example_pred]

'''


all_training_examples = []

for example_num in range(0, len(cur_epi_dates) - (input_hor)): # this is how many times you can "shift and have valid data to pull from"
    print()
    print("Example number: ", example_num)
    # now draw out the adjacency matrix per example
    width_of_adj_per_example = len(unique_geoids)*input_hor
    shift = len(unique_geoids) # number of nodes per day
    adj_per_example = adj_kronecker_whole[ (example_num*shift) : (example_num*shift) + width_of_adj_per_example,
                                           (example_num*shift) : (example_num*shift) + width_of_adj_per_example]

    # now drawing out the train_graph_signal per example

    # get the graph signal corresponding to the example nodes
    train_graph_sig_per_example = {k: train_graph_sig[k] for k in list(train_graph_sig)[(example_num*shift) : (example_num*shift) + width_of_adj_per_example]}

    print("Check: Last few examples in train_graph_sig_per_example: ", {k: train_graph_sig_per_example[k] for k in list(train_graph_sig_per_example)[-5:]})

    # get the graph signal corresponding to the [input_hor : input_hor+pred_hor] set of nodes -- NEEDS CHECKING
    train_graph_sig_per_example_pred = {k: train_graph_sig[k] for k in list(train_graph_sig)[(example_num*shift) + len(unique_geoids)*input_hor : (example_num*shift) + len(unique_geoids)*(input_hor+pred_hor)]}

    print("Check: Last few examples in train_graph_sig_pred: ", {k: train_graph_sig_per_example_pred[k] for k in list(train_graph_sig_per_example_pred)[-5:]}) # working!

    training_example = [adj_per_example, train_graph_sig_per_example, train_graph_sig_per_example_pred]
    all_training_examples.append(training_example)




# Do the training and testing split

# take the last 20% of the the training examples and use them as test examples
all_test_examples = all_training_examples[int(len(all_training_examples)*train_perc):]
# remove those examples from all_training_examples
all_training_examples = all_training_examples[:int(len(all_training_examples)*train_perc)]

print("Number of training examples: ", len(all_training_examples))
print("Number of test examples: ", len(all_test_examples))


Preprocessing data...
Building the kronecker graph...
Check: Last dataframe in graph_kronecker_whole_df:  geoid_o              10030037
geoid_d              11030065
lng_o             1390.824938
lat_o               46.381301
lng_d             1541.601912
lat_d               46.496561
date_x             2019-01-01
visitor_flows        0.109557
pop_flows            0.328222
date_y             2020-06-19
new_confirmed             0.0
infection_gone            0.0
timestep                   10
Name: 157618, dtype: object
Getting the adjacency matrix..
Getting the training graph signal...
0
1
2
3
4
5
6
7
8
9
10
11
Nan value found in train_graph_sig:  2013 nan . Replacing with 0.0 ...
Nan value found in train_graph_sig:  2100 nan . Replacing with 0.0 ...
Nan value found in train_graph_sig:  31007 nan . Replacing with 0.0 ...
Nan value found in train_graph_sig:  38043 nan . Replacing with 0.0 ...
Nan value found in train_graph_sig:  48425 nan . Replacing with 0.0 ...
Check: Last 10 elements 

In [173]:
# Check a single training example
print("Check: Single training example: ")
# print("Adjacency matrix: ", all_training_examples[0][0])
print("Dimensions of adjacency matrix: ", all_training_examples[0][0].shape)
print("Train graph signal: ", all_training_examples[0][1])
print("Dimensions of train graph signal: ", len(all_training_examples[0][1]))
print("Train graph signal prediction: ", all_training_examples[0][2])
print("Dimensions of train graph signal prediction: ", len(all_training_examples[0][2]))


Check: Single training example: 
Dimensions of adjacency matrix:  (9210, 9210)
Train graph signal:  {1001: 10.0, 1003: 5.0, 1005: 2.0, 1007: 6.0, 1009: 2.0, 1011: 5.0, 1013: 9.0, 1015: 2.0, 1017: 9.0, 1019: 0.0, 1021: 6.0, 1023: 1.0, 1025: 3.0, 1027: 1.0, 1029: 0.0, 1031: 2.0, 1033: 5.0, 1035: 3.0, 1037: 0.0, 1039: 1.0, 1041: 1.0, 1043: 4.0, 1045: 4.0, 1047: 15.0, 1049: 4.0, 1051: 11.0, 1053: 0.0, 1055: 1.0, 1057: 2.0, 1059: 6.0, 1061: 0.0, 1063: 3.0, 1065: 3.0, 1067: 0.0, 1069: 7.0, 1071: 3.0, 1073: 53.0, 1075: 0.0, 1077: 4.0, 1079: 1.0, 1081: 5.0, 1083: 9.0, 1085: 8.0, 1087: 4.0, 1089: 11.0, 1091: 0.0, 1093: 0.0, 1095: 8.0, 1097: 26.0, 1099: 6.0, 1101: 98.0, 1103: 34.0, 1105: 2.0, 1107: 2.0, 1109: 9.0, 1111: 0.0, 1113: 10.0, 1115: 0.0, 1117: 6.0, 1119: 1.0, 1121: 4.0, 1123: 2.0, 1125: 41.0, 1127: 7.0, 1129: 3.0, 1131: 6.0, 1133: 1.0, 2013: 0.0, 2016: 0.0, 2020: 5.0, 2050: 0.0, 2070: 0.0, 2090: 0.0, 2100: 0.0, 2110: 0.0, 2122: 3.0, 2130: 0.0, 2150: 0.0, 2170: 0.0, 2180: 1.0, 2185: 0.0

In [174]:
# store the training_data and testing_data in a pickle file
import pickle
with open("training_data.pkl", "wb") as f:
   pickle.dump(all_training_examples, f)

with open("testing_data.pkl", "wb") as f:
   pickle.dump(all_test_examples, f)

In [176]:
# load the pickle files
import pickle
with open("training_data.pkl", "rb") as f:
   training_data = pickle.load(f)

with open("testing_data.pkl", "rb") as f:
    testing_data = pickle.load(f)


In [178]:
print("Undergoing processing to remove nan. If any found will get a print statement below ...")

for data in training_data:
    tr_gr_sig = data[1]
    pred_gr_sig = data[2]

    # Replace nan values with 0.0 in train_graph_sig dictionary
    for key, value in tr_gr_sig.items():
        if np.isnan(value):
            # if found, print statement
            print("Nan value found in train_graph_sig: ", key, value, ". Replacing with 0.0 ...")
            tr_gr_sig[key] = 0.0

    # Replace nan values with 0.0 in pred_graph_sig dictionary
    for key, value in pred_gr_sig.items():
        if np.isnan(value):
            # if found, print statement
            print("Nan value found in pred_graph_sig: ", key, value, ". Replacing with 0.0 ...")
            pred_gr_sig[key] = 0.0
    
    data = (data[0], tr_gr_sig, pred_gr_sig)

for data in testing_data:
    tr_gr_sig = data[1]
    pred_gr_sig = data[2]

    # Replace nan values with 0.0 in train_graph_sig dictionary
    for key, value in tr_gr_sig.items():
        if np.isnan(value):
            # if found, print statement
            print("Nan value found in train_graph_sig: ", key, value, ". Replacing with 0.0 ...")
            tr_gr_sig[key] = 0.0

    # Replace nan values with 0.0 in pred_graph_sig dictionary
    for key, value in pred_gr_sig.items():
        if np.isnan(value):
            # if found, print statement
            print("Nan value found in pred_graph_sig: ", key, value, ". Replacing with 0.0 ...")
            pred_gr_sig[key] = 0.0
    
    data = (data[0], tr_gr_sig, pred_gr_sig)
    

Undergoing processing to remove nan. If any found will get a print statement below ...


In [55]:
from torch_geometric.nn import GCNConv

In [ ]:
# make a custom subclass of Dataset (pytorch class), to the initialization of that class, feed in the traiing_data and testing_data
# inside initialization of the dataset class
# then call the method, get(), if you want to load a sample, what will it return
# at some point the data objects that you have, use the get(index), index = a single training sample (input kronecker graph, output prediction graph signal)
# then create dataloader, give it object of your dataset class 
# for data in dataloader ... loops through the data objects that you have

In [185]:
# GRAPH CONVOLUTIONAL NETWORK BASED ON KRONECKER GRAPH -- TOO MANY PARAMS NOW, TALK TO PEOPLE ABOUT THIS & HOW TO REDUCE

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv

class GraphConvolutionalNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_nodes_kron, num_nodes_pred):
        super(GraphConvolutionalNetwork, self).__init__()
        self.gconv1 = GCNConv(1, 1)
        self.gconv2 = GCNConv(1, 1) # performs graph convolutions on the kronecker graph, then you want to make output predictions
        # self.hidden_to_output = nn.Linear(1, 1)
        self.MLP = nn.Sequential( # takes in 3k^2 parameters to learn
            nn.Linear(num_nodes_kron, 10000),
            # nn.Linear((num_nodes_pred)*input_dim, num_nodes_kron),
            nn.ReLU(),
            nn.Linear(10000, num_nodes_pred*output_dim)
        )
    
    def forward(self, adj_as_edge_index, graph_signal, num_nodes_pred):
        x, edge_index = graph_signal, adj_as_edge_index
        # print(x, edge_index)
        x = x.reshape(1, x.shape[0], x.shape[1])
        x = self.gconv1(x, edge_index)
        # print(x)
        x = torch.relu(x)
        x = self.gconv2(x, edge_index)
        # print(x)
        x = torch.relu(x)
        x = x[-num_nodes_pred:]
        # print(x)
        # print(x.shape)
        x = x.view(x.shape[1])
        # take the output of the kronecker graph convolutions and make predictions
        # only take the last nodes from the final layer to make predictions?
        # x = self.hidden_to_output(x)
        x = self.MLP(x)
        # print(x)
        return x

# Define the model
input_dim = 1  # Number of features per node
hidden_dim = 16  # Embedding dimension
output_dim = 1  # Number of output features per node
num_nodes_kron = training_data[0][0].shape[0]
num_nodes_pred = len(training_data[0][2])

model = GraphConvolutionalNetwork(input_dim, hidden_dim, output_dim, num_nodes_kron, num_nodes_pred)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters:", num_params)

# Define the loss function and optimizer
num_epochs = 20
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

train_losses = []

# Train the model
# if you do it manually, you need to add shuffling

# shuffle the training data randomly
random.shuffle(training_data)
random.shuffle(testing_data)

# batching is not really needed due to nature of graphs, already in "batches"

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    for data in training_data:
        adjacency, graph_signal, target_graph_signal = data
        # print(adjacency.shape, len(graph_signal), len(target_graph_signal))
        
        # Ensure graph_signal is of shape [number_of_nodes, input_dim]
        graph_signal = torch.tensor(list(graph_signal.values()), dtype=torch.float32).view(-1, input_dim)

        # needs to have shape [3070, 1]
        target_graph_signal = torch.tensor(list(target_graph_signal.values()), dtype=torch.float32).view(-1, output_dim)

        # Convert adjacency to edge_index
        adj_as_edge_index = torch.tensor(adjacency.nonzero(), dtype=torch.long)
        
        output = model(adj_as_edge_index, graph_signal, num_nodes_pred) # needs to have shape [3070, 1]
        # output is of shape [3070] turn into shape [3070, 1]
        output = output.view(-1, 1)

        loss = criterion(output, target_graph_signal)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}')

# Test the model
model.eval()
test_loss = 0.0
predictions = []
targets = []
with torch.no_grad():
    for data in testing_data:
        adjacency, graph_signal, target_graph_signal = data
        # print(adjacency.shape, len(graph_signal), len(target_graph_signal))
        
        # Ensure graph_signal is of shape [number_of_nodes, input_dim]
        graph_signal = torch.tensor(list(graph_signal.values()), dtype=torch.float32).view(-1, input_dim)

        # needs to have shape [3070, 1]
        target_graph_signal = torch.tensor(list(target_graph_signal.values()), dtype=torch.float32).view(-1, output_dim)

        # Convert adjacency to edge_index
        adj_as_edge_index = torch.tensor(adjacency.nonzero(), dtype=torch.long)

        out = model(adj_as_edge_index, graph_signal, num_nodes_pred) # needs to have shape [3070, 1]
        # out is of shape [3070] turn into shape [3070, 1]
        out = out.view(-1, 1)
        loss = criterion(out, target_graph_signal)
        test_loss += loss.item()
    test_loss /= len(testing_data)

# Compute metrics
print(f'Test Loss: {test_loss:.4f}')



Number of parameters: 122813074
Epoch 1/20, Train Loss: 1785.7281
Epoch 2/20, Train Loss: 1734.0779
Epoch 3/20, Train Loss: 1596.7501
Epoch 4/20, Train Loss: 1444.2577
Epoch 5/20, Train Loss: 1297.5139
Epoch 6/20, Train Loss: 1173.6266
Epoch 7/20, Train Loss: 1105.9365
Epoch 8/20, Train Loss: 1013.3301
Epoch 9/20, Train Loss: 935.0847
Epoch 10/20, Train Loss: 950.7216
Epoch 11/20, Train Loss: 787.3450
Epoch 12/20, Train Loss: 924.6896
Epoch 13/20, Train Loss: 695.6788
Epoch 14/20, Train Loss: 837.2947
Epoch 15/20, Train Loss: 643.7531
Epoch 16/20, Train Loss: 764.6003
Epoch 17/20, Train Loss: 574.7010
Epoch 18/20, Train Loss: 673.8508
Epoch 19/20, Train Loss: 533.0121
Epoch 20/20, Train Loss: 623.4504
Test Loss: 1584.6447
